# Kleio Walkthrough

## About

Author: Jared Neumann

This package is designed to take a PDF document with or without a text layer, or raw text, and return a complete, corrected version of that text. Text is extracted using common OCR tools, if necessary, and the text is then passed to an LLM. The LLM then makes corrections to each chunk. Additional functions can be called, such as:
- Layout analysis and annotation
- Revised collation (e.g., to eliminate headers and footers, etc.)
- Translation

## Import Statements

In [1]:
# we'll have to set up a duplicate logger in the notebook
import logging

# Get the root logger
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

# Create a StreamHandler for the notebook
stream_handler = logging.StreamHandler()
stream_handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
stream_handler.setFormatter(formatter)

# Add the handler to the logger
logger.addHandler(stream_handler)

# set propagation to false to prevent double logging
logger.propagate = False

from kleio.ocr import *
from kleio.image_utils import *
from kleio.correction import *
from kleio.collation import *
from kleio.translation import *

import matplotlib.pyplot as plt

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
2024-02-01 14:46:15,412 - matplotlib - DEBUG - matplotlib data path: /Users/jneumann/Repos/kleio/.venv/lib/python3.11/site-packages/matplotlib/mpl-data
2024-02-01 14:46:15,415 - matplotlib - DEBUG - CONFIGDIR=/Users/jneumann/.matplotlib
2024-02-01 14:46:15,416 - matplotlib - DEBUG - interactive is False
2024-02-01 14:46:15,416 - matplotlib - DEBUG - platform is darwin
2024-02-01 14:46:15,444 - matplotlib - DEBUG - CACHEDIR=/Users/jneumann/.matplotlib
2024-02-01 14:46:15,446 - matplotlib.font_manager - DEBUG - Using fontManager instance from /Users/jneumann/.matplotlib/fontlist-v330.json


## Getting Raw Text

There are a few allowable file types: PDF with text, PDF without text, images, and plain text. The type is automatically inferred, and the extracted text is returned. A few options for OCR are available.

In [2]:
example_filepath_0 = "../tests/test_input/test_0.pdf"
example_filepath_1 = "../tests/test_input/test_1.jpg"
example_filepath_2 = "../tests/test_input/test_2.pdf"
phil_mag_s1_v1_filepath = "../data/input/phil_mag_s1_v1.pdf"
output_dir = "../data/output"

text = retrieve_text(example_filepath_0)
#text_1 = retrieve_text(example_filepath_1)
#text_2 = retrieve_text(example_filepath_2)
#text_3 = retrieve_text(phil_mag_s1_v1_filepath)

2024-02-01 14:46:15,625 - kleio.ocr - INFO - Retrieving text from ../tests/test_input/test_0.pdf


2024-02-01 14:46:15,625 - kleio.ocr - INFO - Retrieving text from ../tests/test_input/test_0.pdf


2024-02-01 14:46:15,626 - kleio.ocr - INFO - File provided


2024-02-01 14:46:15,626 - kleio.ocr - INFO - File provided


2024-02-01 14:46:15,645 - kleio.ocr - INFO - Getting text from PDF file ../tests/test_input/test_0.pdf


2024-02-01 14:46:15,645 - kleio.ocr - INFO - Getting text from PDF file ../tests/test_input/test_0.pdf


In [3]:
# let's inspect one of the outputs
print(text)
print(len(text["pages"]))
for page in text["pages"]:
    print(page)

{'filename': 'test_0.pdf', 'extension': 'PDF', 'pages': ['[ 2l6 ]\nReceived June 8, 17 dp.\nX X X . A  Letter from  Mr. Lane, Apothe\xad\ncary, \nin Alderfgate-ftreet, to the H\nnourable Henry Cavendifh, \nR. S. on \nthe Solubility o f Iron in fimple \n, by \nthe Intervention of fixed A ir.\nAlderfgate-ftreet, June 5, 1769*\nSi r ,\nRead Nov. 23, F\' g " 1H  E various impregnations of mi- \n7 9 \nJL \nneral waters have always been very \ndifficult to explain : and whoever has read the divers, \nand often contradi&ory reafonings upon the fubjedt, \nmud clearly perceive, that there is ffill room for dif- \ncoveries in this part of natural hiftory*\nYou, Sir, by your accounts of fixed air, and of \nRathbone-place water, related in the laft volume of \nPhilofophical Tranfadtions, have obliged the public \nwith many additional lights on this branch of know* \nledge^ and, from your known accuracy, and diligent \npurfuits in moft philofophical inquiries, the learned \nWorld has great reafon t

## Correcting the Raw Text

In [6]:
import os
correction = get_correction(
    text = text,
    api_key=os.getenv("OPENAI_API_KEY"),
    model_name="gpt-4",
    output_path="../data/output/correction.txt",
    filename=text["filename"],
    filetype="academic journal",
    language="British English",
    date="18th century",
    comments="This is an article from the Philosiphical Transactions of the Royal Society",
)

2024-02-01 14:47:24,207 - kleio.correction - INFO - Getting correction from LLM


2024-02-01 14:47:24,207 - kleio.correction - INFO - Getting correction from LLM


2024-02-01 14:47:24,209 - kleio.llm_utils - INFO - Creating ChatOpenAI prompt...


2024-02-01 14:47:24,209 - kleio.llm_utils - INFO - Creating ChatOpenAI prompt...


2024-02-01 14:47:24,209 - kleio.llm_utils - INFO - Creating OpenAI LLM...


...

2024-02-01 14:47:24,468 - kleio.correction - INFO - Correcting chunks for page 0
2024-02-01 14:47:24,471 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'You are a digitization specialist tasked with replacing poor OCR text with corrected text.'}, {'role': 'user', 'content': 'The text below is a snippet from a digitized text. Your job is to carefully read the text and faithfully correct the OCR. This means keeping in mind the context of the text as you do your job. You have the following additional information about the source text, if available:\n- Source filename: test_0.pdf\n- Source filetype: academic journal\n- OCR software: N/A\n- Image preprocessing software: N/A\n- Publication date: 18th century\n- Language: British English\n- Comments: This is an article from the Philosiphical Transactions of the Royal Society\n\nPlease correct the following text:\n[ 2l6 ]\nR

2024-02-01 14:47:47,461 - kleio.correction - INFO - Correcting chunks for page 1


2024-02-01 14:47:47,461 - kleio.correction - INFO - Correcting chunks for page 1
2024-02-01 14:47:47,467 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'You are a digitization specialist tasked with replacing poor OCR text with corrected text.'}, {'role': 'user', 'content': 'The text below is a snippet from a digitized text. Your job is to carefully read the text and faithfully correct the OCR. This means keeping in mind the context of the text as you do your job. You have the following additional information about the source text, if available:\n- Source filename: test_0.pdf\n- Source filetype: academic journal\n- OCR software: N/A\n- Image preprocessing software: N/A\n- Publication date: 18th century\n- Language: British English\n- Comments: This is an article from the Philosiphical Transactions of the Royal Society\n\nPlease correct the following text:\nr « 7 ]\nb

2024-02-01 14:48:44,135 - kleio.correction - INFO - Correcting chunks for page 2


2024-02-01 14:48:44,135 - kleio.correction - INFO - Correcting chunks for page 2
2024-02-01 14:48:44,141 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'You are a digitization specialist tasked with replacing poor OCR text with corrected text.'}, {'role': 'user', 'content': 'The text below is a snippet from a digitized text. Your job is to carefully read the text and faithfully correct the OCR. This means keeping in mind the context of the text as you do your job. You have the following additional information about the source text, if available:\n- Source filename: test_0.pdf\n- Source filetype: academic journal\n- OCR software: N/A\n- Image preprocessing software: N/A\n- Publication date: 18th century\n- Language: British English\n- Comments: This is an article from the Philosiphical Transactions of the Royal Society\n\nPlease correct the following text:\n[ « » ]\np

## Collating the corrected pages

In [7]:
# As you can see, there are some issues we might still want to resolve
# E.g., words that are split across lines,
# duplicate words on adjacent pages (old texts do this a lot),
# headers and footers that are not part of the main text, etc.
import os
# So, we need to collate the pages to fix these issues
collated_text = collate(
    pages = correction,
    api_key=os.getenv("OPENAI_API_KEY"),
    model_name="gpt-4",
    output_path="../data/output/collation.txt",
    chunk_size=2048,
    remove_headers_and_footers=True,
    add_section_tags=False,
)

2024-02-01 14:49:27,364 - urllib3.connectionpool - DEBUG - https://huggingface.co:443 "HEAD /Xenova/gpt-3.5-turbo/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT3_5Tokenizer'. 
The class this function is called from is 'GPT2TokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2024-02-01 14:49:27,521 - kleio.collation - DEBUG - Chunks: ['[216]\nReceived June 8, 1769.\nXXX. A Letter from Mr. Lane, Apothecary, \nin Aldersgate-street, to the Honourable Henry Cavendish, \nR. S. on \nthe Solubility of Iron in simple \nwater, by \nthe Intervention of fixed Air.\nAldersgate-street, June 5, 1769.\nSir,\nRead Nov. 23, 1769. The various impregnations of mineral \nwaters have always been very \ndifficult

2024-02-01 14:49:27,522 - kleio.llm_utils - INFO - Creating ChatOpenAI prompt...


2024-02-01 14:49:27,522 - kleio.llm_utils - INFO - Creating ChatOpenAI prompt...
2024-02-01 14:49:27,523 - kleio.collation - DEBUG - Prompt: input_variables=['previous_text', 'text'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a detail-oriented content editor who is tasked with formatting OCR text in a particular way for a client.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['previous_text', 'text'], partial_variables={'remove_headers_and_footers': 'True', 'remove_page_numbers': 'True', 'remove_excess_space': 'True', 'remove_empty_lines': 'False', 'remove_line_breaks': 'False', 'remove_word_breaks': 'True', 'add_section_tags': 'False', 'keep_page_breaks': 'True'}, template="INSTRUCTIONS FOR FORMATTING OCR TEXT\nYou will be given a snippet of text from a digitized text as well as the formatted text immaediately before it for context if available. Your job is to carefully read the texts and adjust the format o

2024-02-01 14:49:27,523 - kleio.llm_utils - INFO - Creating OpenAI LLM...


2024-02-01 14:49:27,523 - kleio.llm_utils - INFO - Creating OpenAI LLM...
2024-02-01 14:49:27,524 - httpx - DEBUG - load_ssl_context verify=True cert=None trust_env=True http2=False
2024-02-01 14:49:27,524 - httpx - DEBUG - load_verify_locations cafile='/Users/jneumann/Repos/kleio/.venv/lib/python3.11/site-packages/certifi/cacert.pem'
2024-02-01 14:49:27,532 - httpx - DEBUG - load_ssl_context verify=True cert=None trust_env=True http2=False
2024-02-01 14:49:27,533 - httpx - DEBUG - load_verify_locations cafile='/Users/jneumann/Repos/kleio/.venv/lib/python3.11/site-packages/certifi/cacert.pem'


2024-02-01 14:49:27,539 - kleio.collation - INFO - Collating 1 chunks


2024-02-01 14:49:27,539 - kleio.collation - INFO - Collating 1 chunks
  0%|          | 0/1 [00:00<?, ?it/s]2024-02-01 14:49:27,540 - kleio.collation - DEBUG - Collating chunk: [216]
Received June 8, 1769.
XXX. A Letter from Mr. Lane, Apothecary, 
in Aldersgate-street, to the Honourable Henry Cavendish, 
R. S. on 
the Solubility of Iron in simple 
water, by 
the Intervention of fixed Air.
Aldersgate-street, June 5, 1769.
Sir,
Read Nov. 23, 1769. The various impregnations of mineral 
waters have always been very 
difficult to explain: and whoever has read the diverse, 
and often contradictory reasonings upon the subject, 
must clearly perceive, that there is still room for discoveries in this part of natural history.
You, Sir, by your accounts of fixed air, and of 
Rathbone-place water, related in the last volume of 
Philosophical Transactions, have obliged the public 
with many additional lights on this branch of knowledge, and, from your known accuracy, and diligent 
pursuits in most p

In [8]:
print(collated_text)

Received June 8, 1769.
XXX. A Letter from Mr. Lane, Apothecary, in Aldersgate-street, to the Honourable Henry Cavendish, R. S. on the Solubility of Iron in simple water, by the Intervention of fixed Air.
Aldersgate-street, June 5, 1769.
Sir,
Read Nov. 23, 1769. The various impregnations of mineral waters have always been very difficult to explain: and whoever has read the diverse, and often contradictory reasonings upon the subject, must clearly perceive, that there is still room for discoveries in this part of natural history.
You, Sir, by your accounts of fixed air, and of Rathbone-place water, related in the last volume of Philosophical Transactions, have obliged the public with many additional lights on this branch of knowledge, and, from your known accuracy, and diligent pursuits in most philosophical inquiries, the learned World has great reason to hope for many other new and useful improvements. 
To your judgment therefore, I submit the following experiments; which are intended 

## Translating the corrected, collated text

In [ ]:
TRANSLATION_KWARGS = {
    "target_language": "Spanish",
    "title": "On the Solubility of Iron in Simple, by the Intervention of fixed Air",
    "author": "Mr. Lane",
    "notes": "This is an article from the Philosophical Transactions of the Royal Society from 1770-1800",
}

translation = translate(
    text = collated_text,
    api_key="not-needed",
    llm_provider="openai",
    model_name="mistralai/Mistral-7B-v0.1",
    base_url="http://localhost:1234/v1",
    temperature=0.1,
    chunk_size=2048,
    more_info=TRANSLATION_KWARGS,
)

2024-02-01 14:38:24,052 - httpcore.connection - DEBUG - close.started
2024-02-01 14:38:24,052 - httpcore.connection - DEBUG - close.complete


TypeError: translate() got an unexpected keyword argument 'llm_provider'

In [ ]:
print(translation)

Recibido el 8 de junio de 1770. Sr. Lane, boticario, calle Aldergate, al honorable Henry Cavendish, R.S., sobre la solubilidad del hierro en agua simple, por la intervención del aire fijo. Calle Aldergate, 5 de junio de 1769. Señor, leído el 23 de noviembre de 1769.

Las diversas impregnaciones de las aguas minerales siempre han sido muy difíciles de explicar; y quienquiera que haya leído los diversos y a menudo contradictorios razonamientos sobre este tema, debe percibir claramente que todavía hay espacio para descubrimientos en esta parte de la historia natural.

Usted, señor, con sus relatos sobre el aire fijo, y sobre el agua de Rathbone-place, relatados en el último volumen de las Transacciones Filosóficas, ha obligado al público con muchas luces adicionales sobre esta rama del conocimiento. Y, por su conocida precisión y diligentes investigaciones en la mayoría de las indagaciones filosóficas, el mundo erudito tiene grandes razones para esperar muchas otras mejoras nuevas y útile